# 02 · BLS Baseline (批次處理版) - 使用 01 下載的資料

## 改進內容
- ✅ 讀取 01_tap_download 儲存的 CSV 資料
- ✅ 批量處理 TOI 和 KOI 目標
- ✅ 計算 BLS/TLS 特徵並儲存
- ✅ 支援 Google Drive 和本地儲存

---

In [ ]:
# 環境設定與 NumPy 修復
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📍 偵測到 Google Colab 環境")
    print("🔧 安裝相容版本套件...")
    !pip install -q numpy==1.26.4 pandas astropy scipy'<1.13' matplotlib scikit-learn
    !pip install -q lightkurve astroquery transitleastsquares wotan
    print("✅ 套件安裝完成!")
    print("⚠️ 請現在手動重啟 Runtime: Runtime → Restart runtime")
    print("   然後從下一個 cell 繼續執行")
else:
    print("💻 本地環境，跳過套件安裝")

## 1. 載入 01 下載的資料

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 檢查資料來源
if IN_COLAB:
    # 嘗試從 Google Drive 載入
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        
        # Google Drive 資料路徑
        drive_data_dir = Path('/content/drive/MyDrive/spaceapps-exoplanet/data/latest')
        if drive_data_dir.exists():
            data_dir = drive_data_dir
            print(f"✅ 使用 Google Drive 資料: {data_dir}")
        else:
            data_dir = Path('../data')
            print(f"⚠️ Google Drive 無資料，使用本地: {data_dir}")
    except:
        data_dir = Path('../data')
        print(f"💻 使用本地資料目錄: {data_dir}")
else:
    data_dir = Path('../data')
    print(f"💻 使用本地資料目錄: {data_dir}")

# 載入資料
print("\n📂 載入資料檔案...")

# 載入合併的訓練資料集
supervised_file = data_dir / 'supervised_dataset.csv'
if supervised_file.exists():
    supervised_df = pd.read_csv(supervised_file)
    print(f"   ✅ 載入 supervised_dataset.csv: {len(supervised_df)} 筆")
    
    # 顯示資料摘要
    print(f"\n📊 資料摘要:")
    print(f"   正樣本 (label=1): {(supervised_df['label']==1).sum()} 筆")
    print(f"   負樣本 (label=0): {(supervised_df['label']==0).sum()} 筆")
    
    if 'source' in supervised_df.columns:
        print(f"\n   資料來源分布:")
        for source, count in supervised_df['source'].value_counts().items():
            print(f"   - {source}: {count} 筆")
else:
    print("❌ 找不到 supervised_dataset.csv")
    print("   請先執行 01_tap_download.ipynb 下載資料")
    raise FileNotFoundError("Missing supervised_dataset.csv")

## 2. 篩選可處理的目標

In [ ]:
# 篩選有 TIC 或 KIC ID 的目標
print("\n🔍 篩選可處理的目標...")

# 提取 TIC 目標 (TESS)
tess_targets = supervised_df[supervised_df['target_id'].str.startswith('TIC', na=False)].copy()
print(f"   TESS 目標 (TIC): {len(tess_targets)} 筆")

# 提取 KIC 目標 (Kepler)
kepler_targets = supervised_df[supervised_df['target_id'].str.startswith('KIC', na=False)].copy()
print(f"   Kepler 目標 (KIC): {len(kepler_targets)} 筆")

# 合併目標
processable_targets = pd.concat([tess_targets, kepler_targets])
print(f"\n✅ 總共可處理目標: {len(processable_targets)} 筆")

# 為了示範，先處理前 10 個目標
# 實際執行時可以增加數量或移除限制
SAMPLE_SIZE = 10
sample_targets = processable_targets.head(SAMPLE_SIZE)
print(f"\n📌 本次示範處理前 {SAMPLE_SIZE} 個目標")
print(f"   (可修改 SAMPLE_SIZE 處理更多目標)")

# 顯示樣本
print("\n樣本目標:")
print(sample_targets[['target_id', 'label', 'source', 'period', 'depth']].head())

## 3. 批量下載光曲線並計算 BLS 特徵

In [ ]:
import lightkurve as lk
import time
from typing import Dict, Any, Optional

def process_target_with_bls(
    target_id: str,
    label: int,
    known_period: Optional[float] = None
) -> Optional[Dict[str, Any]]:
    """
    下載光曲線並計算 BLS 特徵
    
    Parameters:
    -----------
    target_id : str
        TIC 或 KIC ID
    label : int
        標籤 (0 或 1)
    known_period : float, optional
        已知週期（如果有）
    
    Returns:
    --------
    dict : 特徵字典，如果失敗則返回 None
    """
    try:
        # 決定任務類型
        if target_id.startswith('TIC'):
            mission = 'TESS'
            search_id = target_id.replace('TIC', 'TIC ')
        elif target_id.startswith('KIC'):
            mission = 'Kepler'
            search_id = target_id.replace('KIC', 'KIC ')
        else:
            return None
        
        # 搜尋光曲線
        search_result = lk.search_lightcurve(
            search_id,
            mission=mission,
            cadence='short',
            author='SPOC' if mission == 'TESS' else None
        )
        
        if len(search_result) == 0:
            print(f"   ⚠️ {target_id}: 無光曲線資料")
            return None
        
        # 下載第一個 sector/quarter
        lc = search_result[0].download()
        
        # 如果是 collection，取第一個
        if hasattr(lc, '__iter__'):
            lc = lc[0]
        
        # 清理和去趨勢
        lc_clean = lc.remove_nans()
        lc_flat = lc_clean.flatten(window_length=401)
        
        # 執行 BLS
        bls = lc_flat.to_periodogram(
            method="bls",
            minimum_period=0.5,
            maximum_period=20.0,
            frequency_factor=5.0
        )
        
        # 提取特徵
        features = {
            'target_id': target_id,
            'label': label,
            'mission': mission,
            'n_points': len(lc_flat.time),
            'bls_period': bls.period_at_max_power.value if hasattr(bls.period_at_max_power, 'value') else bls.period_at_max_power,
            'bls_depth': bls.depth_at_max_power.value if hasattr(bls.depth_at_max_power, 'value') else bls.depth_at_max_power,
            'bls_duration': bls.duration_at_max_power.value if hasattr(bls.duration_at_max_power, 'value') else bls.duration_at_max_power,
            'bls_snr': bls.max_power.value if hasattr(bls.max_power, 'value') else bls.max_power,
            'known_period': known_period,
            'flux_std': np.std(lc_flat.flux.value if hasattr(lc_flat.flux, 'value') else lc_flat.flux),
            'flux_mad': np.median(np.abs(lc_flat.flux.value if hasattr(lc_flat.flux, 'value') else lc_flat.flux - np.median(lc_flat.flux.value if hasattr(lc_flat.flux, 'value') else lc_flat.flux)))
        }
        
        # 計算週期準確度（如果有已知週期）
        if known_period and not pd.isna(known_period) and known_period > 0:
            features['period_accuracy'] = abs(features['bls_period'] - known_period) / known_period
        else:
            features['period_accuracy'] = None
        
        print(f"   ✅ {target_id}: BLS 週期={features['bls_period']:.3f}天, SNR={features['bls_snr']:.2f}")
        return features
        
    except Exception as e:
        print(f"   ❌ {target_id}: 處理失敗 - {str(e)[:50]}")
        return None

In [ ]:
# 批量處理目標
print("\n🚀 開始批量處理目標...\n")

features_list = []
success_count = 0
fail_count = 0

for idx, row in sample_targets.iterrows():
    target_id = row['target_id']
    label = row['label']
    known_period = row.get('period', None)
    
    print(f"處理 {idx+1}/{len(sample_targets)}: {target_id}")
    
    # 處理目標
    features = process_target_with_bls(target_id, label, known_period)
    
    if features:
        features_list.append(features)
        success_count += 1
    else:
        fail_count += 1
    
    # 避免 API 限制
    time.sleep(1)

print(f"\n📊 處理結果:")
print(f"   成功: {success_count} 個")
print(f"   失敗: {fail_count} 個")
print(f"   成功率: {success_count/len(sample_targets)*100:.1f}%")

## 4. 特徵分析與視覺化

In [ ]:
import matplotlib.pyplot as plt

if features_list:
    # 建立特徵 DataFrame
    features_df = pd.DataFrame(features_list)
    print("\n📋 特徵資料摘要:")
    print(features_df[['target_id', 'label', 'bls_period', 'bls_snr', 'bls_depth']].head(10))
    
    # 視覺化：BLS 週期分布
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # 週期分布（按標籤）
    ax1 = axes[0]
    for label in [0, 1]:
        data = features_df[features_df['label'] == label]['bls_period']
        if len(data) > 0:
            ax1.hist(data, bins=20, alpha=0.6, 
                    label=f"Label {label} (n={len(data)})")
    ax1.set_xlabel('BLS 週期 (天)')
    ax1.set_ylabel('數量')
    ax1.set_title('BLS 週期分布')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # SNR 分布（按標籤）
    ax2 = axes[1]
    for label in [0, 1]:
        data = features_df[features_df['label'] == label]['bls_snr']
        if len(data) > 0:
            ax2.hist(data, bins=20, alpha=0.6,
                    label=f"Label {label} (n={len(data)})")
    ax2.set_xlabel('BLS SNR')
    ax2.set_ylabel('數量')
    ax2.set_title('BLS SNR 分布')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計分析
    print("\n📊 特徵統計（按標籤）:")
    print("\n正樣本 (Label=1):")
    pos_features = features_df[features_df['label'] == 1]
    if len(pos_features) > 0:
        print(pos_features[['bls_period', 'bls_snr', 'bls_depth']].describe())
    
    print("\n負樣本 (Label=0):")
    neg_features = features_df[features_df['label'] == 0]
    if len(neg_features) > 0:
        print(neg_features[['bls_period', 'bls_snr', 'bls_depth']].describe())
else:
    print("⚠️ 沒有成功處理的目標")

## 5. 儲存特徵資料

In [ ]:
if features_list:
    # 儲存特徵
    output_dir = Path('../data')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_file = output_dir / 'bls_features.csv'
    features_df.to_csv(output_file, index=False)
    print(f"\n💾 特徵已儲存至: {output_file}")
    print(f"   共 {len(features_df)} 筆特徵")
    
    # 如果在 Colab，也儲存到 Google Drive
    if IN_COLAB:
        try:
            from google.colab import drive
            drive_output = Path('/content/drive/MyDrive/spaceapps-exoplanet/data/latest/bls_features.csv')
            features_df.to_csv(drive_output, index=False)
            print(f"   ✅ 同時儲存到 Google Drive: {drive_output}")
        except:
            print("   ⚠️ 無法儲存到 Google Drive")
    
    # 顯示如何使用這些特徵
    print("\n💡 下一步：")
    print("   1. 增加 SAMPLE_SIZE 處理更多目標")
    print("   2. 執行 03_injection_train.ipynb 訓練分類器")
    print("   3. 使用這些 BLS 特徵進行機器學習")

## 6. 總結

### ✅ 已完成：
1. 成功載入 01_tap_download 的資料
2. 批量下載光曲線並計算 BLS 特徵
3. 儲存特徵供後續訓練使用

### 📊 資料流程：
```
01_tap_download.ipynb
    ↓ (11,979 筆 TOI/KOI 資料)
supervised_dataset.csv
    ↓
02_bls_baseline_batch.ipynb (本筆記本)
    ↓ (下載光曲線 + BLS 分析)
bls_features.csv
    ↓
03_injection_train.ipynb (訓練模型)
```

### 🚀 優化建議：
1. **平行處理**：使用 multiprocessing 加速
2. **錯誤恢復**：儲存中間結果，支援斷點續傳
3. **更多特徵**：加入 TLS、小波變換等特徵
4. **資料增強**：對正樣本進行相位平移增強